# **Install necessary libraries:**

In [27]:
%pip install -q -U google-generativeai semantic_router

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


# Import libraries and configure API key:

In [28]:
import google.generativeai as genai
import os
from semantic_router import Route
API_KEY="Your_api_key"
genai.configure(api_key=API_KEY)

# Set up semantic router:

In [29]:
from semantic_router import RouteLayer
from semantic_router.encoders import HuggingFaceEncoder
encoder = HuggingFaceEncoder()
name: str = "sentence-transformers/all-mpnet-base-v2"
type: str = "huggingface"
encoder.score_threshold=0.3
encoder.name=name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

# Define the chatbot persona:

In [36]:
model = genai.GenerativeModel("gemini-1.5-flash")
context="""You represent Aryaveer in a chatbot, He is a data scientist at JIO. He is Charming and witty , his height is 5'10 , Here are some question answer pair that could help you understand about his life and you have to answer in similar fashion to user queries.\n
What is your name?
You are talkin to me , c'mon now Aryaveerrrr
Whne is your birthday?
23 March
what was your college degree about?
it was better than yours , jk it was in electronics and telecommunications done from kj somaiya , mumbai
who many siblings do you have ?
That's personal dude!
what's your age?
Thank gawd I am not a girl , it's 23
are you male for female?
Human
How many years have you spent in jio?
1
"""

# Define routes for the semantic router: Here one can make routes for different queries depending upon complexity and trigger different retrieval pipelines as per the query.

In [34]:

greeting_route = Route(
    name="greet_user",
    utterances=[
        "hello",
        "hi",
        "hey",
        "Whatsupppppp",
        "good morning",
        "good afternoon",
        "good evening",
    ],
)


user_route = Route(
    name="user_query",
    utterances=[
        "What is your name?",
        "Tell me your birthday?",
    ],
)

general_query_route = Route(
    name="general_query",
    utterances=[
        "Who is the pm of india?",
        "What is the capital of india?",

    ],
)
#    time_route,
routes = [

    greeting_route,
    user_route,
    general_query_route
]
# You can use alternative Encoders, however, in this example we want to showcase a fully-local Semantic Router execution, so we are going to use a
# HuggingFaceEncoder with sentence-transformers/all-MiniLM-L6-v2 (the default) as an embedding model.

score_threshold: float = 0.3
rl = RouteLayer(encoder=encoder, routes=routes)
def semantic_layer(question: str):
  route = rl(question)
  if route.name == "greet_user":
      question += f" Resonse: Hi , you are chatting with aryaveer!"
  elif route.name == "user_query":

    response = model.generate_content(f"""You are aryaveer , a chatbot.You represent a human whose traits are in this context {context} , Answer the user queries sticking to how aryaveer would answer them! or else users might die.
User questions: {question}\n
Always start with , Hie you are talking to me!\n
Do not use any emoji while chatting!!!!
""")

    question += response.text
  else:
      question += f" Resonse: Stop wasting your'e time!"
  return question

**This defines different routes for greeting the user, answering personal questions about Aryaveer, and handling general queries.**

# This creates a route layer and defines a function that takes user input, determines the appropriate route, and generates a response based on the route

In [35]:
question = "Who is the pm of india?"
semantic_layer(question)

"Who is the pm of india? Resonse: Stop wasting your'e time!"

In [37]:
question = "When is your brithday?"
semantic_layer(question)

'When is your brithday?Hie you are talking to me!  23rd March \n'

In [38]:
question = "Hiiii?"
semantic_layer(question)

'Hiiii? Resonse: Hi , you are chatting with aryaveer!'